In [1]:
import os
import optuna
import numpy as np
import pandas as pd
from time import sleep, time
from scipy.special import logit
from scipy.stats import beta
from datetime import datetime

In [2]:
# Generate and save inferred model if does not exist
if not os.path.isfile(f"bayesian_model/glmm.pickle"):
    os.system("python3 infer_bayesian_model.py")

In [3]:
def run_command_gnome(command, title="test", keep_window=False, test=False):
    if keep_window:
        command_suffix = "; $SHELL"
    else:
        command_suffix = ""
        
    if test:
        return f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """
    else:
        os.system(f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """)

In [4]:
def collate_results(sim_name, test="bht", crit_val=0.05):
    try:
        df = pd.read_csv(f"data/simulations/{sim_name}.csv")
        success = (df[f"{test}_pval"] < crit_val).sum()
        fail = (df[f"{test}_pval"] >= crit_val).sum()
        return success, fail
    except FileNotFoundError:
        return 0, 0

In [5]:
def beta_ci(n_success, n_fail, level=0.95, l_perc=None, u_perc=None):
    a = n_success + 1
    b = n_fail+1
    if l_perc==None:
        l_perc = (1-level)/2
    if u_perc==None:
        u_perc = 1-(1-level)/2
    return beta.ppf(l_perc, a, b), beta.ppf(u_perc, a, b)

In [6]:
def generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores, trials_per_sim=100):
    os.system(f"""
        python3 generate_sim_settings.py --p_diff "{p_diff}" --n_raters "{n_raters}" \
        --scores_per_r {scores_per_r} --total_scores {total_scores} --n_sims 1 \
        --trials_per_sim {trials_per_sim} --sim_name {sim_name}
    """)
    return()

In [7]:
def transfer_log(sim_name, power_id, sim_id):
    df = pd.read_csv(f"data/simulations/{sim_name}.csv")
    df.insert(0, "power_id", power_id)
    df["sim_id"] = sim_id
    
    # Overwrite file if start of a new power analysis
    if sim_id == power_id == 0:
        df.to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
    else:
        log = pd.read_csv(f"data/simulations/power_{sim_name}.csv")
        pd.concat([log, df]).to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
        
    return

In [8]:
timeout=600

def objective(trial):
#         Function to optimise
        n_raters = trial.suggest_int("n_raters", 60, 180)

        generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores, trials_per_sim=trials_per_sim)

        # Run simulations
        setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
        last_ran=[0]*N_PROCESSES
        while sum(setting_files)>0:
            for i in range(N_PROCESSES):
                if setting_files[i] and not process_active[i]:
                    source = "source ~/anaconda3/etc/profile.d/conda.sh\n"
                    conda = "conda activate bayes_topic_measures\n"
                    c = f"""python3 perform_sig_test.py --trials_per_sim {trials_per_sim} --process {i} --sim_name {sim_name} --optimal_alloc {OA} --n_runs {N_RUNS}"""
#                     run_command_gnome(source+conda+c, title=f"p{i}", keep_window=False) # Laptop
                    run_command_gnome(c, title=f"p{i}", keep_window=False) # Desktop
                    last_ran[i] = time()
                
                # Removing process running file if timed out
                if time() - last_ran[i] > timeout:
                    os.system(f"rm data/{sim_name}/process_{i}_running")
                    
            sleep(5)
            
            # Calculating early termination
            n_success, n_fail = collate_results(sim_name)
            lower_ci, upper_ci = beta_ci(n_success, n_fail, level=CI_LEVEL)
            
            if 0.9 < lower_ci or upper_ci < 0.9:
                # Wait for all processes to complete
                while sum(process_active) > 0:
                    sleep(1)
                    process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
                
                # Recalculate after waiting
                n_success, n_fail = collate_results(sim_name)
                lower_ci, upper_ci = beta_ci(n_success, n_fail, level=CI_LEVEL)
                if lower_ci < 0.9 < upper_ci:
                    continue
                
                transfer_log(sim_name, power_id, sim_id[0])
                sim_id[0] += 1
                return abs(logit(0.9) - logit((n_success+1)/(n_success+n_fail+2)))
            
            # Refresh setting and process files
            process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
            setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        
        transfer_log(sim_name, power_id, sim_id[0])
        sim_id[0] += 1
        return abs(logit(0.9) - logit((n_success+1)/(n_success+n_fail+2)))

In [9]:
# Settings for hoyle's significance testing
hoyle_total_scores = 50*26
hoyle_p_diff = 0.055
hoyle_scores_per_r = 38

In [10]:
print(datetime.now())

2022-11-07 20:36:47.231670


In [45]:
# Simulation settings
N_PROCESSES = 6
sim_name = "power040"
trials_per_sim = 100
OA = False

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91

power_id, p_diff = 0, 0.040
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
studies[power_id] = optuna.create_study(direction="minimize")
studies[power_id].optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, studies[power_id].best_trial.number,
                  studies[power_id].best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

[I 2022-11-05 18:58:41,204] A new study created in memory with name: no-name-5b737d0a-50d7-4131-b9d6-67448d6ef8ec



    p_diff=0.04, <class 'float'>
    n_raters=165, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.04, <class 'float'>
    n_raters=178, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

...

# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.04, <class 'float'>
    n_raters=115, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.04, <class 'float'>
    n_raters=170, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [42]:
# Simulation settings
N_PROCESSES = 6
sim_name = "power055"
trials_per_sim = 100
OA = False

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91
studies = {}

power_id, p_diff = 0, 0.055
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
studies[power_id] = optuna.create_study(direction="minimize")
studies[power_id].optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, studies[power_id].best_trial.number,
                  studies[power_id].best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

[I 2022-11-04 18:32:26,974] A new study created in memory with name: no-name-4ced23f5-94f7-4530-8e76-79ff72623b3b



    p_diff=0.055, <class 'float'>
    n_raters=123, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.055, <class 'float'>
    n_raters=102, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

...

# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.055, <class 'float'>
    n_raters=116, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.055, <class 'float'>
    n_raters=110, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power055, <class 'str'>
    chain_method=vectorized, <class 'str'>
    append=False, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [12]:
# Simulation settings
N_PROCESSES = 6
sim_name = "power075"
trials_per_sim = 100
OA = False

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91
studies = {}

power_id, p_diff = 0, 0.075
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
studies[power_id] = optuna.create_study(direction="minimize")
studies[power_id].optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, studies[power_id].best_trial.number,
                  studies[power_id].best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

[I 2022-11-06 12:04:30,350] A new study created in memory with name: no-name-093d0e55-3178-424f-af8c-85a3caaf1d1e



    p_diff=0.075, <class 'float'>
    n_raters=179, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power075, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.075, <class 'float'>
    n_raters=89, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power075, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

...

# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.075, <class 'float'>
    n_raters=77, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power075, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.075, <class 'float'>
    n_raters=116, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=power075, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [15]:
# Simulation settings
N_PROCESSES = 6
sim_name = "powerOA040"
trials_per_sim = 100
OA = True

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91

power_id, p_diff = 0, 0.040
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, study.best_trial.number,
                  study.best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

[I 2022-11-08 02:49:24,054] A new study created in memory with name: no-name-6f9043f6-b0b5-4ab1-bc13-09835f644454



    p_diff=0.04, <class 'float'>
    n_raters=108, <class 'int'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=powerOA040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.04, <class 'float'>
    n_raters=131, <class 'int'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=powerOA040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

...


    p_diff=0.04, <class 'float'>
    n_raters=167, <class 'int'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=powerOA040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin


    p_diff=0.04, <class 'float'>
    n_raters=169, <class 'int'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=powerOA040, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

KeyboardInterrupt: 

In [ ]:
# Simulation settings
N_PROCESSES = 6
sim_name = "powerOA055"
trials_per_sim = 100
OA = True

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91
studies = {}

power_id, p_diff = 0, 0.055
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
studies[power_id] = optuna.create_study(direction="minimize")
studies[power_id].optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, studies[power_id].best_trial.number,
                  studies[power_id].best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

In [ ]:
# Simulation settings
N_PROCESSES = 6
sim_name = "powerOA075"
trials_per_sim = 100
OA = True

# p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Optuna settings
N_RUNS = 4
CI_LEVEL = 0.91
studies = {}

power_id, p_diff = 0, 0.075
# for power_id, p_diff in enumerate(np.geomspace(0.02, 0.2, num=10)):
# list is used so that sim_id can be incremented in objective func  
sim_id = [0]
studies[power_id] = optuna.create_study(direction="minimize")
studies[power_id].optimize(objective, n_trials=50)
df = pd.DataFrame([[power_id, p_diff, studies[power_id].best_trial.number,
                  studies[power_id].best_params["n_raters"]]],
                  columns=["power_id", "p_diff", "best_trial", "n_raters"]) # Join power when all is complete
if power_id == 0:
    df.to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)
else:
    log=pd.read_csv(f"data/simulations/optuna_log_{sim_name}.csv")
    pd.concat([log, df]).to_csv(f"data/simulations/optuna_log_{sim_name}.csv", index=False)

In [ ]:
print(datetime.now())

## Significance simulations

In [10]:
def run_command_gnome(command, title="test", keep_window=False, test=False):
    if keep_window:
        command_suffix = "; $SHELL"
    else:
        command_suffix = ""
        
    if test:
        return f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """
    else:
        os.system(f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """)

In [11]:
# Run simulations
def run_simulations(OA=False):
    setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
    process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
    while sum(setting_files)>0:
        for i in range(N_PROCESSES):
            if setting_files[i] and not process_active[i]:
                c = f"""python3 perform_sig_test.py --trials_per_sim {trials_per_sim} --process {i} --optimal_alloc {OA} --sim_name {sim_name}"""
                run_command_gnome(c, title=f"p{i}", keep_window=False)

        sleep(10)
        setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]

In [12]:
# Settings for hoyle's significance testing
hoyle_total_scores = 50*26
hoyle_p_diff = 0.055
hoyle_n_raters = 38
hoyle_scores_per_r = hoyle_total_scores//hoyle_n_raters + 1

In [17]:
# Varying the number of raters
N_PROCESSES = 6
sim_name = "sig_n_raters2"
n_sims = 1_000
trials_per_sim = 1

p_diff = hoyle_p_diff
n_raters = "(20, 150)"
scores_per_r = hoyle_scores_per_r

# Removing all produced files
if os.path.isfile(f"data/{sim_name}/sim_settings.csv"):
    os.system(f"""rm data/{sim_name}/*""")

# Generate settings file
os.system(f"""
    python3 generate_sim_settings.py --n_raters "{n_raters}" --scores_per_r {scores_per_r} \
    --n_sims {n_sims} --trials_per_sim {trials_per_sim} --sim_name {sim_name}
""")

run_simulations()

rm: cannot remove 'data/sig_n_raters2/*': No such file or directory



    p_diff=0.055, <class 'float'>
    n_raters=(20, 150), <class 'tuple'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1000, <class 'int'>
    trials_per_sim=1
    seed=42, <class 'int'>
    sim_name=sig_n_raters2, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [13]:
# Varying the number of raters
N_PROCESSES = 6
sim_name = "sigOA_n_raters2"
n_sims = 1_000
trials_per_sim = 1

p_diff = hoyle_p_diff
n_raters = "(20, 150)"
scores_per_r = hoyle_scores_per_r

# Removing all produced files
if os.path.isfile(f"data/{sim_name}/sim_settings.csv"):
    os.system(f"""rm data/{sim_name}/*""")

# Generate settings file
os.system(f"""
    python3 generate_sim_settings.py --n_raters "{n_raters}" --scores_per_r {scores_per_r} \
    --n_sims {n_sims} --trials_per_sim {trials_per_sim} --sim_name {sim_name}
""")

run_simulations(OA=True)


    p_diff=0.055, <class 'float'>
    n_raters=(20, 150), <class 'tuple'>
    scores_per_r=35, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1000, <class 'int'>
    trials_per_sim=1
    seed=42, <class 'int'>
    sim_name=sigOA_n_raters2, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [18]:
# Varying the topics_per_r and p_diff
N_PROCESSES = 6
sim_name = "sig_p_diff"
n_sims = 1_000
trials_per_sim = 1

p_diff = "(0.04, 0.08)"
scores_per_r = hoyle_scores_per_r
total_scores = hoyle_total_scores

# Removing all produced files
if os.path.isfile(f"data/{sim_name}/sim_settings.csv"):
    os.system(f"""rm data/{sim_name}/*""")

# Generate settings file
os.system(f"""
    python3 generate_sim_settings.py --p_diff "{p_diff}" --scores_per_r "{scores_per_r}" \
    --total_scores {total_scores} --n_sims {n_sims} --trials_per_sim {trials_per_sim} \
    --sim_name {sim_name}
""")

run_simulations()


    p_diff=(0.04, 0.08), <class 'tuple'>
    n_raters=None, <class 'NoneType'>
    scores_per_r=35, <class 'int'>
    total_scores=1300, <class 'int'>
    n_sims=1000, <class 'int'>
    trials_per_sim=1
    seed=42, <class 'int'>
    sim_name=sig_p_diff, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin

In [14]:
# Varying the topics_per_r and p_diff
N_PROCESSES = 6
sim_name = "sigOA_p_diff"
n_sims = 1_000
trials_per_sim = 1

p_diff = "(0.04, 0.08)"
scores_per_r = hoyle_scores_per_r
total_scores = hoyle_total_scores

# Removing all produced files
if os.path.isfile(f"data/{sim_name}/sim_settings.csv"):
    os.system(f"""rm data/{sim_name}/*""")

# Generate settings file
os.system(f"""
    python3 generate_sim_settings.py --p_diff "{p_diff}" --scores_per_r "{scores_per_r}" \
    --total_scores {total_scores} --n_sims {n_sims} --trials_per_sim {trials_per_sim} \
    --sim_name {sim_name}
""")

run_simulations(OA=True)


    p_diff=(0.04, 0.08), <class 'tuple'>
    n_raters=None, <class 'NoneType'>
    scores_per_r=35, <class 'int'>
    total_scores=1300, <class 'int'>
    n_sims=1000, <class 'int'>
    trials_per_sim=1
    seed=42, <class 'int'>
    sim_name=sigOA_p_diff, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to termin